Ahora voy a scrapear información sobre las temporadas

Imports

In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import time

1. Generar URL

In [2]:
seasons_df = pd.read_csv("../data/raw/Used/seasons.csv")

seasons_df["wikipedia_url"] = seasons_df["url"]

2. Scrapear cada temporada

In [4]:
scraped_seasons = []

for index, row in seasons_df.head(10).iterrows():
    season_year = row["year"]
    url = row["wikipedia_url"]

    print(f"Scraping temporada {row["year"]}")

    try:
        headers = {"User-Agent": "UniversityProject/1.0 (Educational Purpose)"}
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            infobox = soup.find("table", {"class": "infobox"})

            if infobox:
                season_data = {"year": season_year}
                rows = infobox.find_all("tr")

                for row_html in rows:
                    header = row_html.find("th")
                    value = row_html.find("td")

                    if header and value:
                        key = header.get_text(strip=True)
                        val = value.get_text(strip=True)
                        

                        if "Drivers" in key and "Champion" in key:
                            # Limpiar flags y referencias [1]
                            driver = val.split("\n")[0].strip()
                            season_data["champion_driver"] = driver
                        
                        # Constructors' Champion / Campeón de Constructores
                        elif "Constructors" in key and "Champion" in key:
                            team = val.split("\n")[0].strip()
                            season_data["champion_team"] = team
                        
                        # Number of races / Número de carreras
                        elif "No. of races" in key or "Races" in key:
                            # Extraer solo el número
                            num = val.split()[0]
                            if num.isdigit():
                                season_data["num_races"] = num
                        
                        # Season starts / Inicio de temporada
                        elif "Season starts" in key or "First race" in key:
                            season_data["calendar_start"] = val
                        
                        # Season ends / Fin de temporada
                        elif "Season ends" in key or "Last race" in key:
                            season_data["calendar_end"] = val
                        
                        # Pole positions leader
                        elif "Pole positions" in key:
                            season_data["pole_positions_leader"] = val.split("\n")[0]
                        
                        # Fastest laps leader
                        elif "Fastest laps" in key:
                            season_data["fastest_laps_leader"] = val.split("\n")[0]
                    
                
                
                scraped_seasons.append(season_data)

            time.sleep(2)

    except Exception as e:
        print(f"Error scraping {url}: {e}")

                
            


                        


Scraping temporada 2009
Scraping temporada 2008
Scraping temporada 2007
Scraping temporada 2006
Scraping temporada 2005
Scraping temporada 2004
Scraping temporada 2003
Scraping temporada 2002
Scraping temporada 2001
Scraping temporada 2000


3. Guardar datos

In [5]:
df_scraped = pd.DataFrame(scraped_seasons)

df_scraped.info()
df_scraped.head()
df_scraped.to_csv("../data/scraped/Prueba01ScrapedSeasons.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame
